# Functional goals for 2019 Q2

This notebook allows interactive exploration of the functionality targeted for the GROMACS master branch in 2019 Q2. 

For named features `fr0`, `fr1`, etcetera, you can build the `acceptance` docker image or download it from docker hub and run with, for example, `docker run --rm -p 8888:8888 gmxapi/acceptance:fr1`

Note that there isn't a great way to use a Jupyter notebook as the front-end to an MPI job. The notebook can be converted to a script and run non-interactively.

    jupyter nbconvert RequiredFunctionality.ipynb --to python
    python RequiredFunctionality.py
    # or
    # mpiexec -n 2 python -m mpi4py RequiredFunctionality.py

Before committing changes to this notebook, clear the output and/or run `python strip_notebook.py RequiredFunctionality.py`

The `has_feature()` expressions allow cells with unimplemented features to raise `gmxapi.exceptions.FeatureError`,
while an automated test can catch the first occurrence of this exception and exit cleanly.

In [ ]:
# Prepare notebook environment.
import gmxapi as gmx
from gmxapi.version import has_feature

# Tests

### fr1: wrap importable Python code.

gmxapi compatible operations are implemented with simple machinery that allows compartmentalized progress on functionality to be highly decoupled from implementing user-facing tools. Tools are provided in `gmx.operation` and demonstrated by implementing `gmx.commandline_operation`.

In [ ]:
has_feature('fr1', enable_exception=True)
# commandline_operation helper creates a set of operations
# that includes the discovery and execution of the program
# named in `executable`.

operation = gmx.commandline_operation(executable='true')
operation.run()
# assert operation.output.returncode.result() == 0
assert operation.output.returncode == 0

operation = gmx.commandline_operation(executable='false')
operation.run()
assert operation.output.returncode == 1


### fr2: output proxy establishes execution dependency


In [ ]:
has_feature('fr2', enable_exception=True)
# A sequence of two shell subcommands writes two lines to a temporary file.
import os
import tempfile
with tempfile.TemporaryDirectory() as directory:
    fh, filename = tempfile.mkstemp(dir=directory)
    os.close(fh)

    line1 = 'first line'
    subcommand = ' '.join(['echo', '"{}"'.format(line1), '>>', filename])
    commandline = ['-c', subcommand]
    filewriter1 = gmx.commandline_operation('bash', arguments=commandline)

    line2 = 'second line'
    subcommand = ' '.join(['echo', '"{}"'.format(line2), '>>', filename])
    commandline = ['-c', subcommand]
    filewriter2 = gmx.commandline_operation('bash', arguments=commandline, input=filewriter1)

    filewriter2.run()
    # Check that the file has the two expected lines
    with open(filename, 'r') as fh:
        lines = [text.rstrip() for text in fh]
    assert len(lines) == 2
    assert lines[0] == line1
    assert lines[1] == line2

### fr3: output proxy can be used as input
<!-- 25 February -->

In [ ]:
has_feature('fr3', enable_exception=True)
import stat
with tempfile.TemporaryDirectory() as directory:
    file1 = os.path.join(directory, 'input')
    file2 = os.path.join(directory, 'output')

    # Make a shell script that acts like the type of tool we are wrapping.
    scriptname = os.path.join(directory, 'clicommand.sh')
    with open(scriptname, 'w') as fh:
        fh.writelines(['#!' + gmx.util.which('bash'),
                       '# Concatenate an input file and a string argument to an output file.',
                       '# Mock a utility with the tested syntax.',
                       '#     clicommand.sh "some words" -i inputfile -o outputfile',
                       'echo $1 | cat - $3 > $5'])
    os.chmod(scriptname, stat.S_IRWXU)

    line1 = 'first line'
    filewriter1 = gmx.commandline_operation(scriptname,
                                        input_files={'-i': os.devnull},
                                        output_files={'-o': file1})

    line2 = 'second line'
    filewriter2 = gmx.commandline_operation(scriptname,
                                        input_files={'-i': filewriter1.output.file['-o']},
                                        output_files={'-o': file2})

    filewriter2.run()
    # Check that the files have the expected lines
    with open(file1, 'r') as fh:
        lines = [text.rstrip() for text in fh]
    assert len(lines) == 1
    assert lines[0] == line1
    with open(file2, 'r') as fh:
        lines = [text.rstrip() for text in fh]
    assert len(lines) == 2
    assert lines[0] == line1
    assert lines[1] == line2

### fr4: dimensionality and typing of named data causes generation of correct work topologies
* gmx.logical_* operations allow optimizable manipulation of boolean values
<!-- 27 February -->

In [ ]:
has_feature('fr4', enable_exception=True)
simulation_input = gmx.read_tpr(initial_tpr)

# Array inputs imply array outputs.
input_array = gmx.modify_input(
    simulation_input, params={'tau-t': [t / 10.0 for t in range(N)]})

md = gmx.mdrun(input_array)  # An array of simulations

rmsf = gmx.commandline_operation(
    'gmx',
    'rmsf',
    input={
        '-f': md.output.trajectory,
        '-s': initial_tpr
    },
    output={'-o': gmx.FileName(suffix='.xvg')})


### fr5: explicit many-to-one or many-to-many data flow
gmx.reduce() helper could simplify expression of operations dependent on gather while allowing under-the-hood optimizations.
<!-- 5 March -->

In [ ]:
has_feature('fr5', enable_exception=True)

output_files = gmx.gather(rmsf.output.file['-o'])
gmx.run()

print('Output file list:')
print(', '.join(output_files.result()))

### fr7: Python bindings for launching simulations

gmx.mdrun uses bindings to C++ API to launch simulations

<!-- 27 February -->

In [ ]:
has_feature('fr7', enable_exception=True)

md = gmx.mdrun(tprfilename)
md.run()
# Note: can't verify that this is accomplished with C++ integration without exploring implementation details.

### fr8: gmx.mdrun understands ensemble work
<!-- 1 March -->

In [ ]:
has_feature('fr8', enable_exception=True)

md = gmx.mdrun([tprfilename, tprfilename])
md.run()
# Maybe assert that two trajectory files with unique filesystem locations are produced?

### fr9: MD plugins
*gmx.mdrun supports interface for binding MD plugins*
(requires interaction with library development)

<!-- 1 March -->

In [ ]:
has_feature('fr9', enable_exception=True)

import sample_restraint

starting_structure = 'input_conf.gro'
topology_file = 'input.top'
run_parameters = 'params.mdp'

initial_tpr = gmx.commandline_operation(
    'gmx',
    'grompp',
    input={
        '-f': run_parameters,
        '-c': starting_structure,
        '-p': topology_file
    },
    output={'-o': gmx.OutputFile('.tpr')})

simulation_input = gmx.read_tpr(initial_tpr.output.file['-o'])

# Prepare a simple harmonic restraint between atoms 1 and 4
restraint_params = {'sites': [1, 4],
                    'R0': 2.0,
                    'k': 10000.0}

restraint = sample_restraint.harmonic_restraint(input=restraint_params)

md = gmx.mdrun(input=simulation_input, potential=sample_restraint)

#md.run()

### fr10: fused operations for use in looping constructs
* gmx.subgraph fuses operations
* gmx.while creates an operation wrapping a dynamic number of iterations of a subgraph

<!-- 10 March -->

In [ ]:
has_feature('fr10', enable_exception=True)

train = gmx.subgraph(variables={'conformation': initial_input})
with train:
    myplugin.training_restraint(
        label='training_potential',
        params=my_dict_params)
    modified_input = gmx.modify_input(
        input=initial_input, structure=train.conformation)
    md = gmx.mdrun(input=modified_input, potential=train.training_potential)
    # Alternate syntax to facilitate adding multiple potentials:
    # md.interface.potential.add(train.training_potential)
    brer_tools.training_analyzer(
        label='is_converged',
        params=train.training_potential.output.alpha)
    train.conformation = md.output.conformation

train_loop = gmx.while_loop(
    operation=train,
    condition=gmx.logical_not(train.is_converged))

### fr11: Python access to TPR file contents
* gmx.read_tpr utility provides access to TPR file contents
* gmx.read_tpr operation produces output consumable by gmx.mdrun
* gmx.mdrun produces gromacs.read_tpr node for tpr filename kwargs

<!-- 1 March -->

In [ ]:
has_feature('fr11', enable_exception=True)

simulation_input = gmx.read_tpr(initial_tpr)
nsteps = simulation_input.params('nsteps')
md = gmx.mdrun(simulation_input)
assert md.output.trajectory.step.result() == nsteps

### fr12: Simulation checkpoint handling

* gmx.mdrun is properly restartable

This should be invisible to the user, and requires introspection and testing infrastructure to properly test (TBD).
<!-- 8 March -->

In [ ]:
has_feature('fr12', enable_exception=True)
from gmxapi import testsupport

simulation_input = gmx.read_tpr(initial_tpr)
md = gmx.mdrun(simulation_input, label='md')
interrupting_context = testsupport.interrupted_md(md)
with interrupting_context as session:
    first_half_md = session.md.run()
md = gmx.mdrun(first_half_md, context=testsupport.inspect)
testsupport.verify_restart(md)

### fr13: ``run`` module function simplifies user experience

* gmx.run finds and runs operations to produce expected output files
* gmx.run handles ensemble work topologies
* gmx.run handles multi-process execution
* gmx.run safety checks to avoid data loss / corruption

<!-- 8 March -->

In [ ]:
has_feature('fr13', enable_exception=True)

md = gmx.mdrun([tprfilename, tprfilename])
gmx.run()

### fr14: Easy access to GROMACS run time parameters

* *gmx.run conveys run-time parameters to execution context*
(requires interaction with library development)

<!-- 15 March -->

In [ ]:
has_feature('fr14', enable_exception=True)

gmx.run(work, tmpi=20, grid=gmx.NDArray([3, 3, 2]), ntomp_pme=1, npme=2, ntomp=1)

### fr15: Simulation input modification
* *gmx.modify_input produces new (tpr) simulation input in data flow operation*
(requires interaction with library development)
* gmx.make_input dispatches appropriate preprocessing for file or in-memory simulation input.

<!-- 15 March -->


In [ ]:
has_feature('fr15', enable_exception=True)

initial_input = gmx.read_tpr([tpr_filename for _ in range(10)])
tau_t = list([i/10. for i in range(10)])
param_sweep = gmx.modify_input(input=initial_input,
                               parameters={ 
                                   'tau_t': tau_t
                               }
                              )
md = gmx.mdrun(param_sweep)
for tau_expected, tau_actual in zip(tau_t, md.output.params['tau_t'].extract()):
    assert tau_expected == tau_actual

### fr16: Create simulation input from simulation output
* *gmx.make_input handles state from checkpoints*
(requires interaction with library development)

<!-- 22 March -->

In [ ]:
has_feature('fr16', enable_exception=True)

initial_input = gmx.read_tpr(tpr_filename)
md = gmx.mdrun(initial_input)
stage2_input = gmx.make_input(topology=initial_input,
                              conformation=md.output,
                              parameters=stage2_params,
                              simulation_state=md.output)
md = gmx.mdrun(stage2_input)
md.run()

### fr17: Prepare simulation input from multiple sources
gmx.write_tpr (a facility used to implement higher-level functionality) merges tpr data (e.g. inputrec, structure, topology) into new file(s)
<!-- 15 March -->

In [ ]:
has_feature('fr17', enable_exception=True)

gmx.fileio.write_tpr(filename=managed_filename, input=stage2_input)
for key, value in gmx.fileio.read_tpr(managed_filename)['input']:
    assert stage2_input[key] == value

### fr18: GROMACS CLI tools receive improved Python-level support over generic commandline_operations
* gmx.tool provides wrapping of unmigrated gmx CLI tools

<!-- 10 March -->

In [ ]:
has_feature('fr18', enable_exception=True)

rmsf = gmx.tool.rmsf(trajectory=md.output.trajectory,
                     structure=initial_tpr)
output_files = gmx.gather(rmsf.output.file).result()

### fr19: GROMACS CLI tools receive improved C++-level support over generic commandline_operations
* gmx.tool uses Python bindings on C++ API for CLI modules (not testable at the UI level)

<!-- 15 March -->

In [ ]:
has_feature('fr19', enable_exception=True)

rmsf = gmx.rmsf(trajectory=md.output.trajectory,
                structure=initial_tpr)

### fr20: Python bindings use C++ API for expressing user interface
*gmx.tool operations are migrated to updated Options infrastructure*
(requires interaction with library development)

<!-- 5 April -->

In [ ]:
has_feature('fr20', enable_exception=True)

analysis = gmx.rmsf(trajectory=md.output.trajectory,
                    topology=initial_input)
file_list = gmx.fileio.write_xvg(analysis.output).result()

### fr21 User insulated from filesystem paths
* gmx.context manages data placement according to where operations run

<!-- 8 March -->

In [ ]:
output_files = gmx.gather(rmsf.output.file).result()
testsupport.verify_unique_xvg_files(output_files)
# Verify completeness of project sustainability goal in code review.

### fr22 MPI-based ensemble management from Python
* gmx.context can own an MPI communicator and run ensembles of simulations.

<!-- 22 March -->

In [ ]:
has_feature('fr22', enable_exception=True)

from mpi4py import MPI
comm_world = MPI.COMM_WORLD

group2 = comm_world.Get_group().Incl([0,1])
ensemble_comm = comm_world.Create_group(group2)

md = gmx.mdrun([tpr_filename for _ in range(2)])

with gmx.get_context(md, communicator=ensemble_comm) as session:
    session.run()

ensemble_comm.Free()

### fr23 Ensemble simulations can themselves use MPI
*GROMACS simulation can use a set of subcommunicators from the comm owned by the client.*
(requires interaction with library development)

<!-- 19 April -->

In [ ]:
has_feature('fr23', enable_exception=True)

from mpi4py import MPI
comm_world = MPI.COMM_WORLD

md = gmx.mdrun([tpr_filename for _ in range(2)])

with gmx.get_context(md, communicator=comm_world) as session:
        session.run()

md = gmx.mdrun([tpr_filename for _ in range(4)])


with gmx.get_context(md, communicator=comm_world) as session:
        session.run()